In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from scipy.io import wavfile
import matplotlib
# matplotlib.use('agg')
from matplotlib import cm
from matplotlib import pyplot as plt
from numpy.lib import stride_tricks
import librosa

In [2]:
""" short time fourier transform of audio signal """
def stft(sig, frameSize, overlapFac=0.9, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))

    # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
    samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)    

In [3]:
""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):        
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[int(scale[i]):])]
        else:
            freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

    return newspec, freqs

In [4]:
""" plot spectrogram"""
def plotstft(samples, samplerate, binsize=400, plotpath=None, colormap="jet"):
    plt.close('all')

    s = stft(samples, binsize)
    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel

#     timebins, freqbins = np.shape(ims)
#     print("timebins: ", timebins)
#     print("freqbins: ", freqbins)
    
#     w = ims.shape[0]/100.0
#     h = ims.shape[1]/100.0

    w = ims.shape[0]/610.0
    h = ims.shape[1]/1250.0
    plt.figure(figsize=(w,h))
    plt.axis('off')
 #   plt.axes([0.,0.,1.,1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    fig = plt.imshow(ims.T, origin="lower", aspect="auto", cmap=colormap, interpolation="nearest")
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    
#    plt.colorbar()

#      plt.xlabel("time (s)")
#      plt.ylabel("frequency (hz)")
#     plt.xlim([0, timebins-1])
#     plt.ylim([0, freqbins])

#    xlocs = np.float32(np.linspace(0, timebins-1, 5))
#    plt.xticks(xlocs, ["%.02f" % l for l in ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate])
#   ylocs = np.int16(np.round(np.linspace(0, freqbins-1, 10)))   
#    yti = ylocs
#   print(freq[20])
#   print(freq[40])
#   print(freq[60])
#   plt.yticks(ylocs, ["%.02f" % freq[i] for i in ylocs])

#   plotpath = 'temp.png'
    if plotpath:
        plt.savefig(plotpath, bbox_inches="tight", pad_inches=0., dpi=1000)
       # image = cv2.imread(plotpath, 1)
        #print(image.shape)
    else:
        plt.show()

    plt.clf()

    return ims

In [5]:
fs = 16000
fsize = 4000
hop = fsize // 2
hop_length = fsize // 40
classes = ['C', 'FA', 'M', 'S']
num_classes = 4

print(fsize)
print(hop)
print(hop_length)
print(classes)
print(num_classes)
print(str((fsize*1000)//fs)+'ms')


4000
2000
100
['C', 'FA', 'M', 'S']
4
250ms


In [6]:
for k in range(0, num_classes):
    y, fs = librosa.load(classes[k] + '.wav', sr = None)
    rng = y.shape[0]//hop - 1
    print(classes[k])

    for i in range(0, rng):
        data = y[i*hop:i*hop+fsize]
        path = './' + str((fsize*1000)//fs) + 'ms/training_set/' + classes[k] + '/' + str(i+1) + '.png'
        ims = plotstft(data, fs, plotpath=path)

C
FA
M
S


<Figure size 11.3311x11.5776 with 0 Axes>

In [7]:
import os
import numpy as np
from shutil import copyfile
for i in range(0, num_classes):
    src = './250ms/training_set/' + classes[i] + '/'
    numImages = len(os.listdir(src))
    indices = np.random.permutation(numImages) + 1
    
    dstTest = './250ms/testing_set/' + classes[i] + '/'
    for j in range(int(0.6 * numImages), int(0.8 * numImages)):
        img = str(indices[j]) + '.png'
        copyfile(src+img, dstTest+img)
        os.remove(src+img)

    dstHold = './250ms/hold_out/' + classes[i] + '/'
    for j in range(int(0.8 * numImages), numImages):
        img = str(indices[j]) + '.png'
        copyfile(src+img, dstHold+img)
        os.remove(src+img)

In [8]:
#y,fs =librosa.load('C.wav', sr=None)

In [9]:
# For individually checking
#ims = plotstft(y[0:8000], fs, plotpath='1.png')